In [81]:
import os.path
import pandapower.networks as pn
import pandapower as pp
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from numpy import flatnonzero as find, ones, zeros, Inf, pi, exp, conj, r_, arange, array
from pypower_pych.makeYbus import makeYbus
from pypower_pych.idx_brch import F_BUS, T_BUS, RATE_A, PF, QF, PT, QT, MU_SF, MU_ST
from pypower_pych.idx_gen import GEN_BUS, PG, QG
from pypower_pych.idx_cost import MODEL, PW_LINEAR, NCOST, POLYNOMIAL
from pypower_pych.totcost import totcost
from pypower_pych.makeSbus import makeSbus
from pypower_pych.idx_bus import BUS_TYPE, REF, VM, VA, MU_VMAX, MU_VMIN, LAM_P, LAM_Q
# import cvxpy as cp
import numpy as np
from scipy.sparse import csr_matrix as sparse
from pypower_pych.idx_bus import PD, QD
from pypower_pych.idx_gen import GEN_BUS, PG, QG, GEN_STATUS

In [82]:
import pandapower as pp
import pandapower.networks as pw
net = pw.case14()
net.gen['vm_pu'] = 1.00
om, ppopt = pp.runopp(net, delta = 1e-16)

In [28]:
import pandapower as pp
net = pp.create_empty_network()

# create buses
bus1 = pp.create_bus(net, vn_kv=110.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)
# bus5 = pp.create_bus(net, vn_kv=110.)

# create 220/110 kV transformer
pp.create_line(net, bus1, bus2, length_km=70., std_type='149-AL1/24-ST1A 110.0')

# create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type="149-AL1/24-ST1A 110.0")
pp.create_line(net, bus4, bus2, length_km=40., std_type="149-AL1/24-ST1A 110.0")

# create loads
pp.create_load(net, bus2, p_mw=60., controllable=False)
pp.create_load(net, bus3, p_mw=70., controllable=False)
pp.create_load(net, bus4, p_mw=25., controllable=False)

# create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=0, max_p_mw=1000)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80, vm_pu=1.00, controllable=True)
g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.00, controllable=True)

costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=20)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

net.line["max_loading_percent"] = 50
om,ppopt = pp.runopp(net, delta=1e-16)

In [83]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,36.719180,23.685221,-4.022315,1.040753
1,28.742791,24.126889,-9.925911,1.015625
2,0.000070,11.545778,-12.689243,1.060000
3,8.495048,8.273014,-10.414843,1.060000


In [84]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.060000,0.000000,-194.330104,-0.000190,36.723777,-9.387659e-02
1,1.040753,-4.022315,-15.019180,-10.985221,38.359589,3.978639e-07
2,1.015625,-9.925911,65.457209,-5.126889,40.574855,3.844322e-07
3,1.014461,-8.664884,47.800000,-3.900000,40.190225,1.198458e-01
4,1.016363,-7.428436,7.600000,1.600000,39.660800,2.075723e-01
5,1.060000,-12.689243,11.199930,-4.045778,39.733707,4.180256e-07
6,1.046347,-11.187884,0.000000,0.000000,40.171513,1.196378e-01
7,1.060000,-10.414843,-8.495048,-8.273014,40.169899,-1.154650e-07
8,1.043699,-12.997160,29.500000,-4.096850,40.166192,1.960301e-01
9,1.039137,-13.232903,9.000000,5.800000,40.317765,3.088292e-01


In [85]:
ppc = om.get_ppc()
baseMVA, bus, gen, branch, gencost = ppc["baseMVA"], ppc["bus"], ppc["gen"], ppc["branch"], ppc["gencost"]
n_b, n_load, n_e, n_line = len(net.bus), len(net.load), len(net.ext_grid), len(branch)

In [86]:
n_b

14

In [87]:
## bounds on optimization vars
x0, xmin, xmax = om.getv()
vv, _, _, _ = om.get_idx()
xmax

array([0.   ,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf, 1.06 , 1.06 , 1.06 , 1.06 ,
       1.06 , 1.06 , 1.06 , 1.06 , 1.06 , 1.06 , 1.06 , 1.06 , 1.06 ,
       1.06 , 3.324, 1.4  , 1.   , 1.   , 1.   , 0.1  , 0.5  , 0.4  ,
       0.24 , 0.24 ])

In [88]:
x0

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 1.06, 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.  , 0.4 , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  ])

In [89]:
## build admittance matrices
Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

/home/ubuntu-h/PycharmProjects/Auto_model_Ps2Dl_v1/pypower_pych/makeYbus.py:49: ComplexWarning: Casting complex values to real discards the imaginary part
  tap[i] = branch[i, TAP]  ## assign non-zero tap ratios
/home/ubuntu-h/PycharmProjects/Auto_model_Ps2Dl_v1/Attvenv/lib/python3.8/site-packages/scipy/sparse/_coo.py:159: ComplexWarning: Casting complex values to real discards the imaginary part
  self.col = np.array(col, copy=copy, dtype=idx_dtype)


In [90]:
## Set the lower and upper bound for all variables
ll, uu = xmin.copy(), xmax.copy()
# print(f'll : {ll}; uu : {uu}')
ll[xmin == -Inf] = -1e10   ## replace Inf with numerical proxies
uu[xmax ==  Inf] =  1e10
Va_refs = bus[bus[:, BUS_TYPE]  == REF, VA] * (pi / 180)
# print(f"Va_refs: {Va_refs}")
ll[vv["i1"]["Va"]:vv["iN"]["Va"]] = -np.ones_like(bus[:, VA]) * (pi / 8) # Va lower bound 赋值
uu[vv["i1"]["Va"]:vv["iN"]["Va"]] = np.ones_like(bus[:, VA]) * (pi / 8) # Va upper bound 赋值
## deal with the Va_refs
ll[vv["i1"]["Va"]:vv["iN"]["Va"]][bus[:, BUS_TYPE]  == REF] = Va_refs  # Va_refs lower bound 赋值
uu[vv["i1"]["Va"]:vv["iN"]["Va"]][bus[:, BUS_TYPE]  == REF] = Va_refs # Va_refs upper bound 赋值
## deal with reactive power, whose ll and uu is 1e9
ll[ll<-1e4] = -100
uu[uu>1e4] = 100

print(f'll"{ll};\n uu:{uu}')

ll"[ 0.00000000e+00 -3.92699082e-01 -3.92699082e-01 -3.92699082e-01
 -3.92699082e-01 -3.92699082e-01 -3.92699082e-01 -3.92699082e-01
 -3.92699082e-01 -3.92699082e-01 -3.92699082e-01 -3.92699082e-01
 -3.92699082e-01 -3.92699082e-01  1.06000000e+00  9.40000000e-01
  9.40000000e-01  9.40000000e-01  9.40000000e-01  9.40000000e-01
  9.40000000e-01  9.40000000e-01  9.40000000e-01  9.40000000e-01
  9.40000000e-01  9.40000000e-01  9.40000000e-01  9.40000000e-01
 -1.00000000e-18 -1.00000000e-18 -1.00000000e-18 -1.00000000e-18
 -1.00000000e-18 -1.00000000e-18 -4.00000000e-01 -1.00000000e-18
 -6.00000000e-02 -6.00000000e-02];
 uu:[0.         0.39269908 0.39269908 0.39269908 0.39269908 0.39269908
 0.39269908 0.39269908 0.39269908 0.39269908 0.39269908 0.39269908
 0.39269908 0.39269908 1.06       1.06       1.06       1.06
 1.06       1.06       1.06       1.06       1.06       1.06
 1.06       1.06       1.06       1.06       3.324      1.4
 1.         1.         1.         0.1        0.5        0

In [91]:
baseMVA

100

In [92]:
v_ll = ll[vv["i1"]["Vm"]:vv["iN"]["Vm"]]
v_uu = uu[vv["i1"]["Vm"]:vv["iN"]["Vm"]]
v_uu

array([1.06, 1.06, 1.06, 1.06, 1.06, 1.06, 1.06, 1.06, 1.06, 1.06, 1.06,
       1.06, 1.06, 1.06])

In [93]:
Pd = bus[:, PD]*0.01
Qd = bus[:, QD]*0.01

In [94]:
il = find(branch[:, RATE_A] < 1e10)
flow_max = (branch[il, RATE_A] / baseMVA)**2
flow_max[flow_max == 0] = 1e5
flow_max

array([9801.+0.j, 9801.+0.j, 9801.+0.j, 9801.+0.j, 9801.+0.j, 9801.+0.j,
       9801.+0.j, 9801.+0.j, 9801.+0.j, 9801.+0.j, 9801.+0.j, 9801.+0.j,
       9801.+0.j, 9801.+0.j, 9801.+0.j, 9801.+0.j, 9801.+0.j, 9801.+0.j,
       9801.+0.j, 9801.+0.j])

In [95]:
on = find(gen[:, GEN_STATUS] > 0)      ## which generators are on?
gbus = gen[on, GEN_BUS]                   ## what buses are they at?

## form net complex bus power injection vector
nb = bus.shape[0]
ngon = on.shape[0]
## connection matrix, element i, j is 1 if gen on(j) at bus i is ON
Cg = sparse((ones(ngon), (gbus, range(ngon))), (nb, ngon))

In [96]:
ipol = find(gencost[:, MODEL] == POLYNOMIAL)   ## poly MW and MVAr costs
First_Or_Con = 4
gencost[ipol, First_Or_Con]

array([0.0430293, 0.25     , 0.01     , 0.01     , 0.01     ])

In [102]:
Pg_lower, Pg_upper = ll[vv["i1"]["Pg"]:vv["iN"]["Pg"]], uu[vv["i1"]["Pg"]:vv["iN"]["Pg"]]
Qg_lower, Qg_upper = ll[vv["i1"]["Qg"]:vv["iN"]["Qg"]], uu[vv["i1"]["Qg"]:vv["iN"]["Qg"]]
Pg_lower * baseMVA

array([-1.e-16, -1.e-16, -1.e-16, -1.e-16, -1.e-16])

In [105]:
lb_update = -1e4
# V_range = 0.02  # 需要进一步设定
V_ll,V_uu = v_ll, v_uu
Pg_lower, Pg_upper = ll[vv["i1"]["Pg"]:vv["iN"]["Pg"]] * baseMVA, uu[vv["i1"]["Pg"]:vv["iN"]["Pg"]] * baseMVA
Qg_lower, Qg_upper = ll[vv["i1"]["Qg"]:vv["iN"]["Qg"]] * baseMVA, uu[vv["i1"]["Qg"]:vv["iN"]["Qg"]] * baseMVA
model = gp.Model("PowerGeneration");model.setParam('NonConvex', 2)
n_bus, n_g = len(net.bus), len(net.gen) + len(net.ext_grid)
V_re = model.addMVar(n_bus, lb=lb_update, vtype=GRB.CONTINUOUS, name='V real')
V_im = model.addMVar(n_bus, lb=lb_update, vtype=GRB.CONTINUOUS, name='V imag')
Pg = model.addMVar(n_g,lb=Pg_lower,  vtype=GRB.CONTINUOUS, name='Pg') # ub=Pg_upper,
Qg = model.addMVar(n_g,   vtype=GRB.CONTINUOUS, name='Qg')# ub=Qg_upper,lb=Qg_lower,
temp1 = model.addMVar(n_bus, lb=lb_update, vtype=GRB.CONTINUOUS, name='Temp 1')
temp2 = model.addMVar(n_bus, lb=lb_update, vtype=GRB.CONTINUOUS, name='Temp 2')
Pg_net = model.addMVar(n_bus, lb=lb_update, vtype=GRB.CONTINUOUS, name='Pg net')
Qg_net = model.addMVar(n_bus, lb=lb_update, vtype=GRB.CONTINUOUS, name='Qg net')
n_l = len(branch)                                                                         # the number of line
Line_temp_1 = model.addMVar(n_l, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 1')    # the shape should be line number
Line_temp_2 = model.addMVar(n_l, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 2')    # the shape should be line number
Line_temp_3 = model.addMVar(n_l, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 3')    # the shape should be line number
Line_temp_4 = model.addMVar(n_l, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 4')    # the shape should be line number

# print(f'Pg_lower:{Pg_lower}\nPg_upper:{Pg_upper}\nQg_lower: {Qg_lower}\n Qg_upper:{Qg_upper}')
# print(f'cost: {gencost[ipol, First_Or_Con]}')
# # V = Vm * exp(1j * Va)
# # V = model.AddGenConstrExp(1j*Va, )
G = np.real(Ybus)
B = np.imag(Ybus)
# print(f'G:{G}\n B:{B}')
G_yf, B_yf = np.real(Yf), np.imag(Yf)
G_yt, B_yt = np.real(Yt), np.imag(Yt)
Temp_c1 = model.addConstr(G@V_re - B@V_im == temp1, 'Temp Constr 1')
Temp_c2 = model.addConstr(G @ V_im + B @ V_re == temp2, 'Temp Constr 2')
Pg_net_c = model.addConstr(Cg @ Pg - Pd == Pg_net, 'Pg net Constr')
Qg_net_c = model.addConstr(Cg @ Qg - Qd == Qg_net, 'Qg net Constr')
# print(f'Pd:{Pd}\n Qd:{Qd}')
CVL_CPL = model.addConstrs((V_re[i] * temp1[i] + V_im[i] * temp2[i] == Pg_net[i] for i in range(n_bus)), 'Cvl P')  # break into different parts
CVL_CPL_Q = model.addConstrs((V_im[i] * temp1[i] - V_re[i] * temp2[i] == Qg_net[i] for i in range(n_bus)), 'Cvl Q')
V_slack = model.addConstr(V_re[0]==1, 'V slack real')
V_slack_a = model.addConstr(V_im[0]==0, 'V slack imag')

V_upper = model.addConstrs((V_re[i] <= 1.06 for i in range(1,n_bus)), 'V upper bound')   #+ V_im[i]**2
V_lower = model.addConstrs((V_re[i]  >= 0.94 for i in range(1,n_bus)), 'V lower bound') #+ V_im[i]**2

# Yf * W * Yf
Line_temp_c_1 = model.addConstr(G_yf@V_re - B_yf@V_im == Line_temp_1, 'Branch Con 1')  # 正向实部电流
Line_temp_c_2 = model.addConstr(G_yf @ V_im + B_yf @ V_re == Line_temp_2, 'Branch Con 2')   # 正向虚部电流
Line_temp_c_3 = model.addConstr(G_yt@V_re - B_yt@V_im == Line_temp_3, 'Branch Con 3')   # 反向实部电流
Line_temp_c_4 = model.addConstr(G_yt @ V_im + B_yt @ V_re == Line_temp_4, 'Branch Con 4') # 反向虚部电流
# Line_c_1 = model.addConstrs((Line_temp_1[i]**2 + Line_temp_2[i]**2 + Line_temp_3[i]**2 + Line_temp_4[i]**2 <= np.real(flow_max[i]) for i in range(n_line)), 'Branch Con 1') # p.u.
# Line_c_2 = model.addConstrs(( <= np.real(flow_max[i]) for i in range(n_line)), 'Branch Con 2') # p.u. np.imag(flow_max) 是零
#gencost[ipol, First_Or_Con]
model.setObjective(gencost[ipol, First_Or_Con]@Pg, GRB.MINIMIZE)
model.optimize()
# model.X[:14]

Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 164 rows, 174 columns and 646 nonzeros
Model fingerprint: 0x4e05f71e
Model has 28 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e-02, 2e-01]
  Bounds range     [1e-16, 1e+04]
  RHS range        [2e-02, 1e+00]
Presolve removed 84 rows and 105 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 105 rows and 112 columns
Presolve time: 0.00s
Presolved: 293 rows, 114 columns, 1000 nonzeros
Presolved model has 52 bilinear constraint(s)
Variable types: 114 continuous, 0 integer (0 binary)

Root relaxation: objective -5.551115e-17, 103 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unex

In [21]:
model.X[:14]

[1.0,
 0.005100875673038342,
 276.64740263168005,
 539.9752378033504,
 4.631632645184473,
 2525.169562924018,
 2643.3219200443255,
 4404.821920044325,
 2324.592965983529,
 2851.492965983529,
 3243.0860942266336,
 3340.221508371691,
 3166.369562924018,
 3635.2073669258702]